In [1]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import precision_recall_curve
import itertools

import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X[self.column]

In [3]:
def negsamp_vectorized_bsearch(pos_inds, n_items, n_samp=32):
    """ Pre-verified with binary search
    `pos_inds` is assumed to be ordered
    """
    raw_samp = np.random.randint(0, n_items - len(pos_inds), size=n_samp)
    pos_inds_adj = pos_inds - np.arange(len(pos_inds))
    ss = np.searchsorted(pos_inds_adj, raw_samp, side='right')
    neg_inds = raw_samp + ss
    return neg_inds

In [4]:
def get_scores(p, test_y):
    precision, recall, thresholds = precision_recall_curve(test_y, p)

    fscore = (2 * precision * recall) / (precision + recall)
    # locate the index of the largest f score
    ix = np.argmax(fscore)
    
    scores = {
        'Threshold': thresholds[ix],
        'F-Score': fscore[ix],
        'Precision': precision[ix],
        'Recall': recall[ix]
    }

    return scores 

In [5]:
def get_scar_res(data, positive_cond, known_labels_ratio=0.8):

    data_P = data[positive_cond]
    NP = int(data_P.shape[0] * known_labels_ratio)
    
    scar_sample = negsamp_vectorized_bsearch(data_P.index, X.shape[0], n_samp=NP)
    data_scar = pd.concat([data_P[:NP], X.iloc[scar_sample]])
    
    #разделим данные на train/test
    X_t, test_X, y_t, test_y = train_test_split(data_scar, data_scar['y'], random_state=6)
    pipeline.fit(X_t, y_t)
    #наши прогнозы для тестовой выборки
    p = pipeline.predict_proba(test_X)[:, 1]
    
    return get_scores(p, test_y)

### Задача
    1. Взять любой набор данных для бинарной классификации 
    2. Сделать feature engineering
    3. Обучить любой классификатор (какой вам нравится)
    4. Далее разделить ваш набор данных на два множества: P (positives) и U (unlabeled). 
        Причем брать нужно не все положительные (класс 1) примеры, а только лишь часть
    5. Применить random negative sampling для построения классификатора в новых условиях
    6. Сравнить качество с решением из пункта 4 (построить отчет - таблицу метрик)
    7. Поэкспериментировать с долей P на шаге 5 (как будет меняться качество модели при уменьшении/увеличении размера P)

In [6]:
items = pd.read_csv('./input/items.csv')
print("Num unique items: {}\nNum unique categories: {}".format(items.shape[0], 
                                                                len(items['item_type'].unique())))
items.head(3)

Num unique items: 10237
Num unique categories: 10


,Name,item_category,item_brand,item_weight,item_type
0,зубная паста лакалют актив 75мл,"Красота, гигиена, бытовая химия",splat,75мл,зубная паста
1,зубная паста лакалют сенситив 75мл,"Красота, гигиена, бытовая химия",splat,75мл,зубная паста
2,зубная паста лесной бальзам ромашка и облепиха...,"Красота, гигиена, бытовая химия",лесной бальзам,NaN,зубная паста


In [7]:
purchases = pd.read_csv('./input/purchases.csv')
print("Num unique users: {}".format(len(purchases['user_id'].unique())))
purchases.columns = ['user_id', 'Name']
purchases.head(3)

Num unique users: 32000


,user_id,Name
0,ed6b1aaf-21df-5b75-9b7f-ed67926cd17c,"шоколад ""alpen gold"" белый с миндалем и кокосо..."
1,ba82ad84-3a19-5a91-8e1e-7fd87628afb4,пюре тема говядина с гречкой с 8 месяцев
2,74a2856d-f0ec-59a6-89f3-1f80b294e852,колбаса микоян сервелат кремлевский варено-коп...


In [8]:
purchases = pd.merge(purchases, items)
purchases.head(3)

,user_id,Name,item_category,item_brand,item_weight,item_type
0,ed6b1aaf-21df-5b75-9b7f-ed67926cd17c,"шоколад ""alpen gold"" белый с миндалем и кокосо...","Хлеб, сладости, снеки",alpen gold,90г,шоколад
1,b4a10859-3f8c-5dc1-8d5d-5977f9aa8bde,"шоколад ""alpen gold"" белый с миндалем и кокосо...","Хлеб, сладости, снеки",alpen gold,90г,шоколад
2,464053f2-ead4-500e-8486-9d5d66c1bbd7,"шоколад ""alpen gold"" белый с миндалем и кокосо...","Хлеб, сладости, снеки",alpen gold,90г,шоколад


In [9]:
purchases['y'] = purchases['item_type'].apply(lambda x: 1 if x=='чай' else 0, 1)

In [10]:
X = purchases.groupby(['user_id']).agg({'Name': lambda x: list(x), 
                                        'y': lambda x: max(x)})
X['user_id'] = [i for i in X.index.values]
X['user_id'] = X.index.values
X.columns = ['last_purchases', 'y', 'user_id']
X = X[['user_id', 'last_purchases', 'y']]
X.index = range(len(X))
X.head(3)

,user_id,last_purchases,y
0,00002f01-66e4-5ab8-8d1a-1562a4ddd418,[зубная паста splat stress off антистресс 75мл...,0
1,0000fed8-b063-51ef-8ca4-c42c5bd022ad,[шоколад schogetten black & white молочный с к...,0
2,0004cfe8-bcb2-5a2c-904b-643e0469cbe3,"[шоколад воздушный темный 85г, сыр белебеевски...",0


In [11]:
X['y'].value_counts()

0    30640
1     1360
Name: y, dtype: int64

In [12]:
X['last_purchases'] = X['last_purchases'].apply(lambda x: " ".join(x), 1)

In [13]:
#разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(X, X['y'], random_state=6)

In [14]:
model = DecisionTreeClassifier(max_depth=None, max_features=None, 
                                   criterion='gini', class_weight='balanced')

pipeline = Pipeline([('last_purchases_selector', FeatureSelector(column='last_purchases')), 
                     ('last_purchases_tfidf', TfidfVectorizer()), 
                     ('clf', model)])

#обучим наш пайплайн
pipeline.fit(X_train, y_train)

Pipeline(steps=[('last_purchases_selector',
                 FeatureSelector(column='last_purchases')),
                ('last_purchases_tfidf', TfidfVectorizer()),
                ('clf', DecisionTreeClassifier(class_weight='balanced'))])

In [15]:
#наши прогнозы для тестовой выборки
preds = pipeline.predict_proba(X_test)[:, 1]
get_scores(preds, y_test)

{'Threshold': 1.0,
 'F-Score': 0.9134078212290503,
 'Precision': 0.9108635097493036,
 'Recall': 0.9159663865546218}

In [16]:
get_scar_res(X, X['y']==1, 0.5)

{'Threshold': 1.0,
 'F-Score': 0.9905362776025236,
 'Precision': 0.9936708860759493,
 'Recall': 0.9874213836477987}

In [17]:
get_scar_res(X, X['y']==1, 0.8)

{'Threshold': 1.0,
 'F-Score': 0.9843749999999999,
 'Precision': 1.0,
 'Recall': 0.9692307692307692}

In [18]:
get_scar_res(X, X['y']==1, 0.9)

{'Threshold': 1.0,
 'F-Score': 0.987012987012987,
 'Precision': 0.9967213114754099,
 'Recall': 0.977491961414791}